In [2]:
# Cargar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
import mlflow
import mlflow.sklearn


In [4]:
# Leer datos
df = pd.read_csv("data/diabetes.csv")

In [5]:
# identifica las columnas que tiene valores 0
(df==0).sum()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [6]:
# Reemplazar ceros en columnas específicas por NaN
cols_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin']
# No se consideran algunas, por ejemplo Pregnancies (por qué si es posible que se tengan cero embarazos)

# Reemplazar ceros por NaN para poder tratarlos como datos faltantes
for col in cols_to_clean:
    df[col] = df[col].replace(0, np.nan)

# Reemplazar NaN con la moda (valor más frecuente) de cada columna
for col in cols_to_clean:
    moda = df[col].mode()[0]# buscar el método más común para determinar la moda de una columna
    df[col] = df[col].fillna(moda)


In [7]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,105.0,33.6,0.627,50,1
1,1,85.0,66.0,29.0,105.0,26.6,0.351,31,0
2,8,183.0,64.0,32.0,105.0,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,105.0,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,105.0,30.1,0.349,47,1


In [8]:
# Separar datos
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# agregar el traking uri de mlflow
mlflow.set_tracking_uri("http://localhost:9090/")
# agregar set_experiment con nombre: ClasificadorDemoDiabetes
mlflow.set_experiment(experiment_name="ClasificadorDemoDiabetes")

2025/05/21 21:28:32 INFO mlflow.tracking.fluent: Experiment with name 'ClasificadorDemoDiabetes' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1747880912757, experiment_id='1', last_update_time=1747880912757, lifecycle_stage='active', name='ClasificadorDemoDiabetes', tags={}>

In [11]:
# Entrenamiento y registro con MLflow

# usar los parámetros del archivo logreg_variaciones_educativas.csv
# para los parámtros C, max_iter, solver y penalty de LogisticRegresion
# se debe genear un run por cada fila del archivo, usando sus parámetros
# Analizar que usar

# Leer el archivo de parámetros
df_variaciones = pd.read_csv("data/logreg_variaciones_educativas.csv")

# agregar la línea del start_run
for index, row in df_vaciaciones.iterrows():
    with mlflow.start_run(run_name=row["run_id"]):
        # El pipeline usar StandarScaler para que todos los valore numéricos estén en la misma escala
        # no cambiar dicha línea
        # LogisticRegresion deben variar sus parámetros en cada iteración.
        pipeline = Pipeline([
                ("scaler", StandardScaler()),
                ("clf", LogisticRegression(
                    C=row["logreg_C"],
                    max_iter=int(row["logreg_max_iter"]),
                    solver=row["solver"],
                    penalty=row["penalty"]
                ))
            ])

        # Entrenar y evaluar
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)

        # recuerde que log_param y log_metric debe ir guardando los valores
        # dependiendo del run correspondiente de cada fila
        mlflow.log_param("logreg_C", row["logreg_C"])
        mlflow.log_param("logreg_max_iter", row["logreg_max_iter"])
        mlflow.log_param("solver", row["solver"])
        mlflow.log_param("penalty", row["penalty"])

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)

        # Guardar el pipeline completo
        mlflow.sklearn.log_model(pipeline, "modelo_pipeline")

        print(f"Run {row['run_id']} registrado en MLflow")
        print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f}")

Run practica_01 registrado en MLflow
Accuracy: 0.7208 | Precision: 0.6034
Run practica_02 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
Run practica_03 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
Run practica_04 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
Run practica_05 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
Run practica_06 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
Run practica_07 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
Run practica_08 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
Run practica_09 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
Run practica_10 registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
